### Create scenarios and upgrade congested transmission line

In [1]:
from powersimdata import Scenario
scenario = Scenario('')
scenario.set_builder(grid_model="usa_tamu", interconnect="Eastern")
scenario.set_name("MISONC_MTEP16_MEP", "training")
scenario.set_time("2016-01-01 00:00:00","2016-01-31 23:00:00","24H")
scenario.set_base_profile("demand", "vJan2021")
scenario.set_base_profile("hydro", "vJan2021")
scenario.set_base_profile("solar", "vJan2021")
scenario.set_base_profile("wind", "vJan2021")

# Upgrade the congested branch 68928 from 13.66MVA RateA to 236MW, which is a 17.28-time increase. Impedance will be automatically reduced proportionally.
scenario.change_table.scale_branch_capacity(branch_id = {68928: 17.28})

grid = scenario.get_grid()
ct = scenario.get_ct()

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
--> Summary
# Existing study
test
# Available profiles
demand: vJan2021
hydro: vJan2021
solar: vJan2021
wind: vJan2021


### Use the change table to modify conditions

In [4]:
scenario.change_table.scale_plant_capacity(
    "solar", zone_name={"Arkansas": 5, "Louisiana": 2.5})
scenario.change_table.scale_plant_capacity(
    "wind", zone_name={"Iowa": 1.5, "Minnesota Southern": 2})
scenario.change_table.scale_branch_capacity(
    zone_name={"Iowa": 2, "Minnesota Southern": 2})

# add AC line between two busses in IA
scenario.change_table.add_branch(
    [{"capacity": 236, "from_bus_id": 54070, "to_bus_id": 54057}])

# add DC line between central IA and Chicago, IL
scenario.change_table.add_dcline(
    [{"capacity": 2000, "from_bus_id":55119, "to_bus_id": 1213}])

# add a solar plant in AK, a coal plant in IA and a natural gas plant in LA
scenario.change_table.add_plant(
    [{"type": "solar", "bus_id": 1899, "Pmax": 75},
     {"type": "coal", "bus_id": 54070, "Pmin": 25, "Pmax": 750, "c0": 1800, "c1": 30, "c2": 0.0025},
     {"type": "ng", "bus_id": 60911, "Pmax": 75, "c0": 900, "c1": 30, "c2": 0.0015}])

# add a new bus, and a new one-way DC line connected to this bus
scenario.change_table.add_bus(
	[{"lat": 35, "lon": -90, "zone_id": 42, "baseKV": 500}])
scenario.change_table.add_dcline(
	[{"from_bus_id": 1899, "to_bus_id": 70048, "Pmin": 0, "Pmax": 200}])

### Review the scenario and change table, the launch simulation

In [5]:
grid = scenario.get_grid()
ct = scenario.get_ct()
scenario.print_scenario_info()

--------------------
SCENARIO INFORMATION
--------------------
plan: MISONC_MTEP16_MEP
name: training
state: create
grid_model: usa_tamu
interconnect: Eastern
base_demand: vJan2021
base_hydro: vJan2021
base_solar: vJan2021
base_wind: vJan2021
change_table: Yes
start_date: 2016-01-01 00:00:00
end_date: 2016-01-31 23:00:00
interval: 24H
engine: REISE.jl


In [ ]:
scenario.create_scenario()

print(scenario.state.name)
scenario.state.print_scenario_status()

In [ ]:
scenario.state.prepare_simulation_input()

In [ ]:
resp = scenario.launch_simulation(solver="glpk")

In [ ]:
scenario.check_progress()

### Retrieve simulation output again

In [ ]:
from powersimdata.scenario.scenario import Scenario
scenario = Scenario("training") 
scenario.state.print_scenario_info() 
ct = scenario.state.get_ct() 
grid = scenario.state.get_grid()
congu = scenario.state.get_congu()                                                     
congl = scenario.state.get_congl()
congu_yearly = (congu.sum(axis = 0)[congu.sum(axis = 0)!=0]).sort_values(ascending=False)
congl_yearly = (congl.sum(axis = 0)[congl.sum(axis = 0)!=0]).sort_values(ascending=False)
congl[68928]
congl[68928].astype(bool).sum(axis=0)